### Deliverable 3: Optimize Model### Deliverable 1: Preprocessing the Data for a Neural Network

#### Pre-Processing Data

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd
import tensorflow as tf
import os

In [2]:
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns.
# In addition to dropping 'EIN' and 'NAME' as in other notebook, also
# dropping 'STATUS' and 'SPECIAL_CONSIDERATIONS' column since those
# features may be adding additional noise.
application_df = application_df.drop(['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'], 1)

C:\Users\chris\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Determine which values to replace if counts are less than ...?
replace_application = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_counts[classification_counts < 1500].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [10]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

C:\Users\chris\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop(columns=['IS_SUCCESSFUL'])


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

#### Compile, Train and Evaluate the Model

#### Optmization Attempt 1: Increase Number of Nodes in Hidden Layers

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 120
hidden_nodes_layer2 = 80

nn_opt1 = tf.keras.models.Sequential()

# First hidden layer
nn_opt1.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_opt1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_opt1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_opt1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               4920      
                                                                 
 dense_1 (Dense)             (None, 80)                9680      
                                                                 
 dense_2 (Dense)             (None, 1)                 81        
                                                                 
Total params: 14,681
Trainable params: 14,681
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn_opt1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [17]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period = 5)

In [18]:
# Train the model
fit_model = nn_opt1.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 681us/step - loss: 0.5682 - accuracy: 0.7225
Epoch 2/100
804/804 [==============================] - 1s 662us/step - loss: 0.5547 - accuracy: 0.7310
Epoch 3/100
804/804 [==============================] - 1s 703us/step - loss: 0.5519 - accuracy: 0.7324
Epoch 4/100
804/804 [==============================] - 1s 660us/step - loss: 0.5499 - accuracy: 0.7312
Epoch 5/100
804/804 [==============================] - 1s 666us/step - loss: 0.5487 - accuracy: 0.7333
Epoch 6/100
804/804 [==============================] - 1s 687us/step - loss: 0.5481 - accuracy: 0.7327
Epoch 7/100
804/804 [==============================] - 1s 682us/step - loss: 0.5475 - accuracy: 0.7346
Epoch 8/100
804/804 [==============================] - 1s 679us/step - loss: 0.5461 - accuracy: 0.7347
Epoch 9/100
804/804 [==============================] - 1s 658us/step - loss: 0.5458 - accuracy: 0.7335
Epoch 10/100
804/804 [==============================] - 1s 679us/step - l

804/804 [==============================] - 1s 747us/step - loss: 0.5341 - accuracy: 0.7405
Epoch 80/100
804/804 [==============================] - 1s 694us/step - loss: 0.5336 - accuracy: 0.7410
Epoch 81/100
804/804 [==============================] - 1s 721us/step - loss: 0.5340 - accuracy: 0.7414
Epoch 82/100
804/804 [==============================] - 1s 757us/step - loss: 0.5341 - accuracy: 0.7411
Epoch 83/100
804/804 [==============================] - 1s 743us/step - loss: 0.5342 - accuracy: 0.7408
Epoch 84/100
804/804 [==============================] - 1s 673us/step - loss: 0.5339 - accuracy: 0.7418
Epoch 85/100
804/804 [==============================] - 1s 723us/step - loss: 0.5345 - accuracy: 0.7407
Epoch 86/100
804/804 [==============================] - 1s 777us/step - loss: 0.5337 - accuracy: 0.7413
Epoch 87/100
804/804 [==============================] - 1s 735us/step - loss: 0.5338 - accuracy: 0.7415
Epoch 88/100
804/804 [==============================] - 1s 694us/step - loss:

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_opt1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5673 - accuracy: 0.7256 - 215ms/epoch - 801us/step
Loss: 0.5673072934150696, Accuracy: 0.7255976796150208


#### Optmization Attempt 2: Add an Additional Hidden Layer

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 120
hidden_nodes_layer2 = 80
hidden_nodes_layer3 = 30

nn_opt2 = tf.keras.models.Sequential()

# First hidden layer
nn_opt2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_opt2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_opt2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn_opt2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_opt2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 120)               4920      
                                                                 
 dense_4 (Dense)             (None, 80)                9680      
                                                                 
 dense_5 (Dense)             (None, 30)                2430      
                                                                 
 dense_6 (Dense)             (None, 1)                 31        
                                                                 
Total params: 17,061
Trainable params: 17,061
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compile the model
nn_opt2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [23]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period = 5)

In [24]:
# Train the model
fit_model = nn_opt2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 772us/step - loss: 0.5699 - accuracy: 0.7220
Epoch 2/100
804/804 [==============================] - 1s 722us/step - loss: 0.5537 - accuracy: 0.7315
Epoch 3/100
804/804 [==============================] - 1s 756us/step - loss: 0.5512 - accuracy: 0.7325
Epoch 4/100
804/804 [==============================] - 1s 777us/step - loss: 0.5496 - accuracy: 0.7333
Epoch 5/100
804/804 [==============================] - 1s 755us/step - loss: 0.5482 - accuracy: 0.7337
Epoch 6/100
804/804 [==============================] - 1s 727us/step - loss: 0.5481 - accuracy: 0.7340
Epoch 7/100
804/804 [==============================] - 1s 741us/step - loss: 0.5464 - accuracy: 0.7355
Epoch 8/100
804/804 [==============================] - 1s 919us/step - loss: 0.5461 - accuracy: 0.7361
Epoch 9/100
804/804 [==============================] - 1s 798us/step - loss: 0.5458 - accuracy: 0.7356
Epoch 10/100
804/804 [==============================] - 1s 733us/step - l

804/804 [==============================] - 1s 796us/step - loss: 0.5329 - accuracy: 0.7420
Epoch 80/100
804/804 [==============================] - 1s 770us/step - loss: 0.5329 - accuracy: 0.7411
Epoch 81/100
804/804 [==============================] - 1s 944us/step - loss: 0.5326 - accuracy: 0.7411
Epoch 82/100
804/804 [==============================] - 1s 778us/step - loss: 0.5327 - accuracy: 0.7418
Epoch 83/100
804/804 [==============================] - 1s 810us/step - loss: 0.5326 - accuracy: 0.7421
Epoch 84/100
804/804 [==============================] - 1s 737us/step - loss: 0.5325 - accuracy: 0.7419
Epoch 85/100
804/804 [==============================] - 1s 749us/step - loss: 0.5322 - accuracy: 0.7416
Epoch 86/100
804/804 [==============================] - 1s 794us/step - loss: 0.5329 - accuracy: 0.7420
Epoch 87/100
804/804 [==============================] - 1s 795us/step - loss: 0.5322 - accuracy: 0.7425
Epoch 88/100
804/804 [==============================] - 1s 748us/step - loss:

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_opt2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5870 - accuracy: 0.7248 - 206ms/epoch - 770us/step
Loss: 0.5869910717010498, Accuracy: 0.724781334400177


#### Optmization Attempt 3: Modify Nodes Across All Hidden Layers and Change Activation Function in Third Hidden Layer

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn_opt3 = tf.keras.models.Sequential()

# First hidden layer
nn_opt3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_opt3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_opt3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn_opt3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_opt3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                3280      
                                                                 
 dense_8 (Dense)             (None, 30)                2430      
                                                                 
 dense_9 (Dense)             (None, 10)                310       
                                                                 
 dense_10 (Dense)            (None, 1)                 11        
                                                                 
Total params: 6,031
Trainable params: 6,031
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Compile the model
nn_opt3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [28]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [29]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period = 5)

In [30]:
# Train the model
fit_model = nn_opt3.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 702us/step - loss: 0.5668 - accuracy: 0.7224
Epoch 2/100
804/804 [==============================] - 1s 704us/step - loss: 0.5539 - accuracy: 0.7307
Epoch 3/100
804/804 [==============================] - 1s 763us/step - loss: 0.5510 - accuracy: 0.7310
Epoch 4/100
804/804 [==============================] - 1s 709us/step - loss: 0.5493 - accuracy: 0.7341
Epoch 5/100
804/804 [==============================] - 1s 746us/step - loss: 0.5489 - accuracy: 0.7343
Epoch 6/100
804/804 [==============================] - 1s 699us/step - loss: 0.5475 - accuracy: 0.7344
Epoch 7/100
804/804 [==============================] - 1s 773us/step - loss: 0.5464 - accuracy: 0.7345
Epoch 8/100
804/804 [==============================] - 1s 700us/step - loss: 0.5458 - accuracy: 0.7344
Epoch 9/100
804/804 [==============================] - 1s 738us/step - loss: 0.5453 - accuracy: 0.7344
Epoch 10/100
804/804 [==============================] - 1s 713us/step - l

804/804 [==============================] - 1s 759us/step - loss: 0.5350 - accuracy: 0.7420
Epoch 80/100
804/804 [==============================] - 1s 931us/step - loss: 0.5352 - accuracy: 0.7409
Epoch 81/100
804/804 [==============================] - 1s 907us/step - loss: 0.5347 - accuracy: 0.7405
Epoch 82/100
804/804 [==============================] - 1s 777us/step - loss: 0.5349 - accuracy: 0.7407
Epoch 83/100
804/804 [==============================] - 1s 780us/step - loss: 0.5353 - accuracy: 0.7408
Epoch 84/100
804/804 [==============================] - 1s 688us/step - loss: 0.5349 - accuracy: 0.7420
Epoch 85/100
804/804 [==============================] - 1s 732us/step - loss: 0.5343 - accuracy: 0.7411
Epoch 86/100
804/804 [==============================] - 1s 732us/step - loss: 0.5349 - accuracy: 0.7411
Epoch 87/100
804/804 [==============================] - 1s 747us/step - loss: 0.5350 - accuracy: 0.7404
Epoch 88/100
804/804 [==============================] - 1s 729us/step - loss:

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_opt3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5575 - accuracy: 0.7250 - 189ms/epoch - 706us/step
Loss: 0.5575289130210876, Accuracy: 0.7250145673751831


#### Export Optimized Model to HDF5 File

In [32]:
# Export our model to HDF5 file
# Since Optmization Attempt #1 had the highest accuracy, will save that model
nn_opt1.save("AlphabetSoupCharity_Optimization.h5")